# Übertragung des neuen Kategoriensystems aus der Excel-Tabelle in das tex-Dokument der Diss

In diesem ipynb wird das Kategoriensystem, das in der Exceltabelle "Kategoriensystem_neu/Kategoriensystem_neu_komplett.xlsx" beschrieben ist, in Latex-Tabellen übertragen.

In [4]:
# Einlesen der Bibliotheken:
import numpy as np 
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from JanPromoBib_Auswertung_v5_DissFinal import *

In [95]:
# Vorbereitungen und allgemeine Angaben: 

# neu auf Mac:
Ordner_Quelle = "/Users/janheysel/Documents/Promotion/EvaDiss_DatenBestehend"
Ordner_Ziel_Daten = "/Users/janheysel/Documents/Promotion/Diss schreiben/FF1_Tabellen_NeuesKatSystem"
StammDateiname_out = "KatNeu_" # Dateiname = StammDateiname_out + code + ".tex"
Dateiname_in = "Kategoriensystem_FINAL.xlsx"

codes = ["M1", "M2", "M3", "M4", "S1", "S2", "S3", "PR1", "PR2", "PR3"]
codes_SPR = ["S1", "S2", "S3", "PR1", "PR2", "PR3"]
codes_neu = ["M4", "S1", "S2", "S3", "PR1", "PR2", "PR3"]

# Für Anhang:
spalten_droppen = ["Titel MaxQDA (nicht in Haupttext)"]
spaltenbreiten_cm = [8,6,0.5,0.5,0.5]

# Number of digits to round the percentage numbers of the pre and post values in the latex document
nd = 1

In [97]:
fragen = {"M1": "Was ist ein Modell?", 
          "M2": "Zu welchem Ziel oder Zweck entwickeln Forschende Modelle?", 
          "M3": "Wie kann man ein na.wi. Modell überprüfen?",
          "M4": "Was kann daraus schließen, wenn Prognosen aus einem Modell gut mit Messwerten aus Beobachtungen in der Natur übereinstimmen?",
          "S1": "Was ist eine numerische Simulation?",
          "S2": "Wozu nutzt man numerische Simulationen?",
          "S3": "Welche Grundschritte muss man machen, um eine numerische Simulation durchzuführen?",
          "PR1": "Was ist das Hauptziel, warum man ein Peer Review Verfahren macht?",
          "PR2": "Aus welchen Schritten besteht ein Peer Review Verfahren?",
          "PR3": "Welche Eigenschaften des Peer Review Verfahrens dienen dazu, die Ziele des Peer Review Verfahrens zu erreichen?"}

In [99]:
# Funktion, um einen String zu wrappen (Zeilenumbruch in Zelle und Breite angeben):
def wrap_string_atwidth(s, breite):
    """s: die Eingabe, die gewrappt wird, falls es sich um einen String handelt
    breite: die Breite der Spalte als String für Latex. Z.B. breite = '5cm' 
    """
    
    if isinstance(s, str):
        return '\parbox[t]{' + breite + '}{' + s + '}'
    return s

# Vollständige Tabellen für Anhang

In [101]:


for code in codes_SPR: 
    
    df = pd.read_excel(Ordner_Quelle + "/" + Dateiname_in, sheet_name=code+"z")
    
    # index und Diskussion dropen:
    df = df.drop(columns=spalten_droppen)
    if code == "M1":
        df = df.drop(columns=["enthaltene Kategorien"])
    else:
        a = 42 # sinnfreies else - es musste aber ein else hinter das if...
        
    
    # Sonderzeichen ersetzen: 
    df = df.replace(to_replace= r"&", value="und", regex=True) # for regex see https://www.w3schools.com/python/python_regex.asp
    df = df.replace(to_replace= r"%", value="Prozent", regex=True)    
    df.replace('\u00A6', '|', regex=True, inplace=True)
    df.replace('\u20AC', 'EUR', regex=True, inplace=True)
    
    # drop rows with nan (in M2):
    df = df.dropna(subset=['Post', 'Pre'], how='all')
    
    # calculate pre and post to relative Numbers:
    N = 142
    df["Pre"] = df["Pre"] / N * 100
    df["Pre"] = df["Pre"].round(nd)
    df["Post"] = df["Post"] / N * 100
    df["Post"] = df["Post"].round(nd)
        
    # Angaben für Latex definieren: 
    dateiname_latex = StammDateiname_out + "out2" + code + ".tex"
    pfad = Ordner_Ziel_Daten + "/" + dateiname_latex
    
    my_label = "tab:FF1_KatSysNeu_" + code
    my_caption = "Kategoriensystem zu Frage " + code + ": " + fragen[code]
    
    my_column_format = ""

    # Spaltenbreiten für die Einträge hinzufügen: 
    Spalten = df.columns.values #"Fragen " + code
    i = 0
    for spalte in Spalten:
        print(i)
        breite = str(spaltenbreiten_cm[i]) + "cm"
        df[spalte] = df[spalte].apply(wrap_string_atwidth, args=(breite,))
        
        my_column_format = my_column_format + "p{" + str(spaltenbreiten_cm[i]) + "cm}"
        i = i+1
   
    
    # Tabellen als Latex ausgeben: 
    df_style = df.style.format(hyperlinks = "latex", precision=1)
    df_style = df_style.hide(axis="index")
    df_style.to_latex(pfad, label = my_label, position = "ht", caption = my_caption, column_format = my_column_format, environment = "longtable", hrules=True)
    
   
    
    # change "continued on next page" to "Fortsetzung auf der nächsten Seite":
    file_path = pfad 
    search_phrase = "Continued on next page"
    replacement_phrase = "Fortsetzung auf der nächsten Seite"

    # Read the contents of the file
    with open(file_path, 'r', encoding='utf-8') as file:
        file_content = file.read()

    # Replace the search phrase with the replacement phrase
    updated_content = file_content.replace(search_phrase, replacement_phrase)

    # Write the updated content back to the file
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(updated_content)
        
    print(code)
    print(pfad)
    
    
    
# Achtung: an die Tabellen, die im Latex-Dokument der Diss eingebunden werden, hänge ich manuell noch ein "_Diss" vor das ".tex", 
#          damit evtl. manuell gemachte Verbesserungen (z.B. Sonderzeichen, die das Kompilieren behindert haben) nicht wieder überschrieben werden.

0
1
2
3
4
S1
/Users/janheysel/Documents/Promotion/Diss schreiben/FF1_Tabellen_NeuesKatSystem/KatNeu_out2S1.tex
0
1
2
3
4
S2
/Users/janheysel/Documents/Promotion/Diss schreiben/FF1_Tabellen_NeuesKatSystem/KatNeu_out2S2.tex
0
1
2
3
4
S3
/Users/janheysel/Documents/Promotion/Diss schreiben/FF1_Tabellen_NeuesKatSystem/KatNeu_out2S3.tex
0
1
2
3
4
PR1
/Users/janheysel/Documents/Promotion/Diss schreiben/FF1_Tabellen_NeuesKatSystem/KatNeu_out2PR1.tex
0
1
2
3
4
PR2
/Users/janheysel/Documents/Promotion/Diss schreiben/FF1_Tabellen_NeuesKatSystem/KatNeu_out2PR2.tex
0
1
2
3
4
PR3
/Users/janheysel/Documents/Promotion/Diss schreiben/FF1_Tabellen_NeuesKatSystem/KatNeu_out2PR3.tex


# Gekürzte Tabellen für Haupttext

In [115]:
spaltenbreiten_cm_Haupttext = [8,0.5,0.5]

for code in codes: 

    # Daten aus Excel-Tabelle einlesen: 
    if code in codes_SPR:
        code_sheet = code+"z"
        spalten_droppen_Haupttext = ["Titel MaxQDA (nicht in Haupttext)", "\textbf{Beispiele}", "\textbf{Niv.}"]
    else: 
        code_sheet = code
        spalten_droppen_Haupttext = ["Titel MaxQDA (nicht in Haupttext)", "Beispiele", "Niv."]
    print(code, code_sheet)
    
    df = pd.read_excel(Ordner_Quelle + "/" + Dateiname_in, sheet_name=code_sheet)
    
    # index und Diskussion dropen:
    if code in codes_SPR:
        df = df[['Definition', "Pre", "Post"]]
    else:
        df = df.drop(columns=spalten_droppen_Haupttext)
    if code == "M1":
        df = df.drop(columns=["enthaltene Kategorien"])
    else:
        a = 42 # sinnfreies else - es musste aber ein else hinter das if...
        
    
    # Sonderzeichen ersetzen: 
    df = df.replace(to_replace= r"&", value="und", regex=True) # for regex see https://www.w3schools.com/python/python_regex.asp
    df = df.replace(to_replace= r"%", value="Prozent", regex=True)    
    df.replace('\u00A6', '|', regex=True, inplace=True)
    df.replace('\u20AC', 'EUR', regex=True, inplace=True)
    
    # drop rows with nan (in M2):
    df = df.dropna(subset=['Post', 'Pre'], how='all')
    
    # calculate pre and post to relative Numbers:
    N = 142
    df["Pre"] = df["Pre"] / N * 100
    df["Pre"] = df["Pre"].round(nd)
    df["Post"] = df["Post"] / N * 100
    df["Post"] = df["Post"].round(nd)
        
    # Angaben für Latex definieren: 
    dateiname_latex = StammDateiname_out + code + "_kurz_original" + ".tex"
    pfad = Ordner_Ziel_Daten + "/" + dateiname_latex
    
    my_label = "tab:FF1_KatSys_kurz_" + code
    my_caption = "Übersicht zum Kategoriensystem zu Oberkategorie " + code + "\\\ (" + fragen[code]+ ")"
    
    my_column_format = ""

    # Spaltenbreiten für die Einträge hinzufügen: 
    Spalten = df.columns.values #"Fragen " + code
    i = 0
    for spalte in Spalten:
        my_column_format = my_column_format + "p{" + str(spaltenbreiten_cm_Haupttext[i]) + "cm}"
        i = i+1
   
    
    # Tabellen als Latex ausgeben: 
    df_style = df.style.format(hyperlinks = "latex", precision=1)
    df_style = df_style.hide(axis="index")
    df_style.to_latex(pfad, label = my_label, position = "h", caption = my_caption, column_format = my_column_format)#, environment = "longtable", hrules=True)
        
    print(code)

M1 M1
M1
M2 M2
M2
M3 M3
M3
M4 M4
M4
S1 S1z
S1
S2 S2z
S2
S3 S3z
S3
PR1 PR1z
PR1
PR2 PR2z
PR2
PR3 PR3z
PR3
